In [2]:
import pandas as pd

In [3]:
players = pd.read_csv('./Data/players.csv')
matches = pd.read_csv('./Data/match.csv')
heroes = pd.read_csv('./Data/hero_names.csv')
items = pd.read_csv('./Data/item_ids.csv')    

In [4]:
hero_lookup = dict(zip(heroes['hero_id'], heroes['localized_name']))
hero_lookup[0] = 'Unknown'
players['hero'] = players['hero_id'].apply(lambda _id: hero_lookup[_id])

In [5]:
item_lookup = dict(zip(items['item_id'], items['item_name']))
item_lookup[0] = 'Unknown'

def find_item(_id):
    return item_lookup.get(_id, 'u_' + str(_id))

players['item_0'] = players['item_0'].apply(find_item)
players['item_1'] = players['item_1'].apply(find_item)
players['item_2'] = players['item_2'].apply(find_item)
players['item_3'] = players['item_3'].apply(find_item)
players['item_4'] = players['item_4'].apply(find_item)
players['item_5'] = players['item_5'].apply(find_item)

In [6]:
player_heroes = pd.get_dummies(players['hero'])

In [7]:
item0 = pd.get_dummies(players['item_0'].fillna(0))
item1 = pd.get_dummies(players['item_1'].fillna(0))
item2 = pd.get_dummies(players['item_2'].fillna(0))
item3 = pd.get_dummies(players['item_3'].fillna(0))
item4 = pd.get_dummies(players['item_4'].fillna(0))
item5 = pd.get_dummies(players['item_5'].fillna(0))

In [8]:
player_items = item0 \
    .add(item1, fill_value=0) \
    .add(item2, fill_value=0) \
    .add(item3, fill_value=0) \
    .add(item4, fill_value=0) \
    .add(item5, fill_value=0)

In [8]:
radiant_cols = list(map(lambda s: 'radiant_' + s, player_heroes.columns.values))
dire_cols = list(map(lambda s: 'dire_' + s, player_heroes.columns.values))

In [11]:
radiant_items_cols = list(map(lambda s: 'radiant_' + str(s), player_items.columns.values))
dire_items_cols = list(map(lambda s: 'dire_' + str(s), player_items.columns.values))

In [17]:
from os.path import isfile

X = None

# if isfile('mapped_match_hero_item.csv'):
#    X = pd.read_csv('mapped_match_hero_item.csv')
# else:

radiant_heroes = []
dire_heroes = []
radiant_items = []
dire_items = []

for _index in players.groupby('match_id').groups.items():
    radiant_heroes.append(player_heroes.iloc[_index[1][:5]].sum())
    dire_heroes.append(player_heroes.iloc[_index[1][5:]].sum())
    radiant_items.append(player_items.iloc[_index[1][:5]].sum())
    dire_items.append(player_items.iloc[_index[1][5:]].sum())

radiant_heroes = pd.DataFrame(radiant_heroes, columns=radiant_cols)
dire_heroes = pd.DataFrame(dire_heroes, columns=dire_cols)
radiant_items = pd.DataFrame(radiant_items, columns=radiant_items_cols)
dire_items = pd.DataFrame(dire_items, columns=dire_items_cols)
X = pd.concat([radiant_heroes, radiant_items, dire_heroes, dire_items], axis=1)
X.to_csv('mapped_match_hero_item.csv', index=False)

KeyboardInterrupt: 

In [ ]:
X.head()
y = matches['radiant_win'].apply(lambda win: 1 if win else 0)
classes = ['Dire Win', 'Radiant Win']

In [ ]:
_ = pd.Series(y).apply(lambda i: classes[i]).value_counts().plot('bar')